In [17]:
import aiohttp
import asyncio

class URLParse:

    def __init__(self, url=None):
        self.url = url

    @staticmethod
    def get_name(splitted, data_dict):
        idx = splitted.index('<div class="instrumentHead">') + 1
        b = splitted[idx]
        c = b.find('\"name\">')
        d = b.find('</h1>')
        name = b[c + 7:d - 1]
        data_dict['name'] = name

    @staticmethod
    def get_pair_id(raw_site, data_dict):
        idx = raw_site.index(r'pair-id="') + 9
        d = raw_site[idx:].index(r'"')
        pair_id = raw_site[idx:idx + d]
        data_dict['pair_id'] = pair_id

    @staticmethod
    def get_price(splitted, data_dict):
        idx = splitted.index('<div class="top bold inlineblock">') + 1
        b = splitted[idx]
        c = b.find('dir="ltr">')
        d = b.find('</span>')
        price = b[c + 10:d]
        data_dict['price'] = price.replace('.', '').replace(',', '.')

    @staticmethod
    def get_currency(splitted, data_dict):
        idx = splitted.index('<div class="bottom lighterGrayFont arial_11">') + 2
        b = splitted[idx]
        c = b.find('Цена в <span class=\'bold\'>')
        d = b[c + 26:].find('</span>')
        currency = b[c + 26:c + 26 + d]
        data_dict['currency'] = currency

    @staticmethod
    def get_ma(splitted, data_dict):
        idx = splitted.index('<div class="newTechStudiesRight instrumentTechTab" id="techStudiesInnerWrap">') + 2
        b = splitted[idx]
        c = b.index('<i id="maBuy">(')
        d = b[c + 15:].index(')</i>')
        ma_buy = int(b[c + 15:c + 15 + d])
        data_dict['1hr maBuy'] = ma_buy
        c = b.index('<i id="maSell">(')
        d = b[c + 16:].index(')</i>')
        ma_sell = int(b[c + 16:c + 16 + d])
        data_dict['1hr maSell'] = ma_sell
        data_dict['1hr MA'] = ma_buy - ma_sell

    @staticmethod
    def get_ti(splitted, data_dict):
        idx = splitted.index('<div class="newTechStudiesRight instrumentTechTab" id="techStudiesInnerWrap">') + 3
        b = splitted[idx]
        c = b.index('<i id="tiBuy">(')
        d = b[c + 15:].index(')</i>')
        ti_buy = int(b[c + 15:c + 15 + d])
        data_dict['1hr tiBuy'] = ti_buy
        c = b.index('<i id="tiSell">(')
        d = b[c + 16:].index(')</i>')
        ti_sell = int(b[c + 16:c + 16 + d])
        data_dict['1hr tiSell'] = ti_sell
        data_dict['1hr TI'] = ti_buy - ti_sell
        data_dict['1hr SUM'] = data_dict['1hr MA'] + data_dict['1hr TI']

    @staticmethod
    def get_ma_dim(raw_site, data_dict, timings, i):
        idx = raw_site.index('<i id="maBuy">(') + 15
        d = raw_site[idx:].index(')</i>')
        ma_buy = int(raw_site[idx:idx + d])
        data_dict[timings[i][0] + ' maBuy'] = ma_buy
        idx = raw_site.index('<i id="maSell">(') + 16
        d = raw_site[idx:].index(')</i>')
        ma_sell = int(raw_site[idx:idx + d])
        data_dict[timings[i][0] + ' maSell'] = ma_sell
        data_dict[timings[i][0] + ' MA'] = ma_buy - ma_sell

    @staticmethod
    def get_ti_dim(raw_site, data_dict, timings, i):
        idx = raw_site.index('<i id="tiBuy">(') + 15
        d = raw_site[idx:].index(')</i>')
        ti_buy = int(raw_site[idx:idx + d])
        data_dict[timings[i][0] + ' tiBuy'] = ti_buy
        idx = raw_site.index('<i id="tiSell">(') + 16
        d = raw_site[idx:].index(')</i>')
        ti_sell = int(raw_site[idx:idx + d])
        data_dict[timings[i][0] + ' tiSell'] = ti_sell
        data_dict[timings[i][0] + ' TI'] = ti_buy - ti_sell
        data_dict[timings[i][0] + ' SUM'] = data_dict[timings[i][0] + ' MA'] + data_dict[timings[i][0] + ' TI']

    async def parse(self):
        async with aiohttp.ClientSession() as s:
            data_dict = {'name': None, 'pair_id': None, 'price': None, 'currency': None,
                         'Overall SUM': None, '1hr SUM': None, '5hr SUM': None, '1d SUM': None, '1w SUM': None,
                         '1m SUM': None,
                         '1hr maBuy': None, '1hr maSell': None, '1hr MA': None, '1hr tiBuy': None, '1hr tiSell': None,
                         '1hr TI': None,
                         '5hr maBuy': None, '5hr maSell': None, '5hr MA': None, '5hr tiBuy': None, '5hr tiSell': None,
                         '5hr TI': None,
                         '1d maBuy': None, '1d maSell': None, '1d MA': None, '1d tiBuy': None, '1d tiSell': None,
                         '1d TI': None,
                         '1w maBuy': None, '1w maSell': None, '1w MA': None, '1w tiBuy': None, '1w tiSell': None,
                         '1w TI': None,
                         '1m maBuy': None, '1m maSell': None, '1m MA': None, '1m tiBuy': None, '1m tiSell': None,
                         '1m TI': None,
                         }
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                              ' Chrome/92.0.4515.131 Safari/537.36'}
            headers2 = {
                'accept': '*/*',
                'authority': 'ru.investing.com',
                'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
                'content-type': 'application/x-www-form-urlencoded',
                'origin': 'https://ru.investing.com',
                'sec-ch-ua': '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"',
                'sec-ch-ua-mobile': '?0',
                'sec-fetch-dest': 'empty',
                'sec-fetch-mode': 'cors',
                'sec-fetch-site': 'same-origin',
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                              ' Chrome/92.0.4515.131 Safari/537.36',
                'x-requested-with': 'XMLHttpRequest',
            }
            timings = {0: ['5hr', '18000'], 1: ['1d', '86400'], 2: ['1w', 'week'], 3: ['1m', 'month']}
            result = s.get(ref, headers=headers)
            raw_site = result.text
            splitted = tuple(raw_site.splitlines())

            self.get_name(splitted, data_dict)
            self.get_pair_id(raw_site, data_dict)
            self.get_price(splitted, data_dict)
            self.get_currency(splitted, data_dict)
            self.get_ma(splitted, data_dict)
            self.get_ti(splitted, data_dict)

            for i in range(4):
                body = 'pairID=' + data_dict['pair_id'] + '&period=' + timings[i][1] + '&viewType=normal'
                result = s.post('https://ru.investing.com/instruments/Service/GetTechincalData', headers=headers2, data=body)
                raw_site = result.text
                self.get_ma_dim(raw_site, data_dict, timings, i)
                self.get_ti_dim(raw_site, data_dict, timings, i)

            data_dict['Overall SUM'] = data_dict['1hr SUM'] + data_dict['5hr SUM'] + data_dict['1d SUM'] + data_dict[
                '1w SUM'] + data_dict['1m SUM']
        return data_dict

In [12]:
import aiohttp
import asyncio

        for ref in market.ref_list:
            a.url = ref
            temp_dict.append(a.parse(s))


    final_dict = dict.fromkeys(list(temp_dict[0]), [])
    for bar in list(temp_dict[0]):
        listt = []
        for foo in range(len(temp_dict)):
            listt.append(temp_dict[foo][bar])
        final_dict[bar] = listt

    table_all = pd.DataFrame(data=final_dict)
    print('Данные технического анализа по ' + market.name + ' успешно загружены')

    end = time.time()
    print(f'Runtime of the program is {end - start}')
    print(datetime.datetime.now())

    return table_all

In [18]:
import time
import datetime
from support_module import RefList
from support_module import Table
import aiohttp
import asyncio

if __name__ == '__main__':
    if (datetime.datetime.today().weekday() != 5) and (datetime.datetime.today().weekday() != 6):
        temp_dict = []
        world = RefList('ММВБ+РТС')
        table = None
        #table = get_table(world)
        a = URLParse()

        loop = asyncio.get_event_loop()

        coroutines = [a.parse() for ref in world.ref_list]

        results = loop.run_until_complete(asyncio.gather(*coroutines))
        
        """
        final_dict = dict.fromkeys(list(temp_dict[0]), [])
        for bar in list(temp_dict[0]):
            listt = []
            for foo in range(len(temp_dict)):
                listt.append(temp_dict[foo][bar])
            final_dict[bar] = listt

        table_all = pd.DataFrame(data=final_dict)
        print('Данные технического анализа по ' + market.name + ' успешно загружены')

        end = time.time()
        print(f'Runtime of the program is {end - start}')
        print(datetime.datetime.now())"""

<ipython-input-18-3f3f0fd4d0de>:10: RuntimeWarning: coroutine 'URLParse.parse' was never awaited
  temp_dict = []


RuntimeError: This event loop is already running

In [2]:
import pandas as pd

In [10]:
table

<coroutine object get_table at 0x0000017F57B4E1C0>